In [27]:
from feature.make_dataset import make_data
from feature.after_dataset import accumulate
from feature.moving_average import moving_average
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


In [28]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [29]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)
train_rol = moving_average(train_input_list)
test_rol = moving_average(test_input_list)

In [30]:
train = train.reset_index()
test = test.reset_index()
train_rol = train_rol.reset_index()
test_rol = test_rol.reset_index()

In [31]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적총광량')

In [32]:
train = pd.concat([train, train_rol], axis=1)
test = pd.concat([test, test_rol], axis=1)

In [33]:
drop_feature = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량', 'index']
train = train.drop(drop_feature, axis=1)
test = test.drop(drop_feature, axis=1)


In [39]:
X = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
y = train['predicted_weight_g']
from sklearn.model_selection import StratifiedKFold, KFold

skf = KFold(n_splits=5, shuffle=True, random_state=404)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

CAT_model = {}

for f in range(5):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    CAT = CatBoostRegressor(verbose=500, early_stopping_rounds=50, task_type="GPU", random_seed=113)
    CAT.fit(x_train, y_train)

    y_pred = CAT.predict(x_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    print(f"{f + 1} Fold RMSE = {rmse}")
    CAT_model[f] = CAT
    print(f'================================================================================\n\n')


submit = test[['DAT', 'predicted_weight_g']]
x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
for fold in range(5):
    submit['predicted_weight_g'] += CAT_model[fold].predict(x_test)/5
all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
for idx, test_path in enumerate(all_target_list):
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
    submit_df.to_csv(test_path, index=False)

====================================1============================================
Learning rate set to 0.037997
0:	learn: 41.5876795	total: 10.6ms	remaining: 10.6s
500:	learn: 3.8953458	total: 4.94s	remaining: 4.92s
999:	learn: 3.1296310	total: 9.75s	remaining: 0us
1 Fold RMSE = 8.186903900991904


====================================2============================================
Learning rate set to 0.037997
0:	learn: 39.9878956	total: 11.1ms	remaining: 11.1s
500:	learn: 4.8482479	total: 4.98s	remaining: 4.96s
999:	learn: 4.2272455	total: 9.76s	remaining: 0us
2 Fold RMSE = 8.615328689744814


====================================3============================================
Learning rate set to 0.037997
0:	learn: 41.1692615	total: 10.4ms	remaining: 10.3s
500:	learn: 4.5017835	total: 5s	remaining: 4.98s
999:	learn: 4.0742393	total: 9.55s	remaining: 0us
3 Fold RMSE = 6.95923532490345


====================================4============================================
Learning rate set to 0